# Check the Chapter 7 IRF

Piers 

In [ ]:
#from ar6.twolayermodel import TwoLayerModel
import numpy as np
import matplotlib.pyplot as pl
from tqdm import tqdm
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from netCDF4 import Dataset
import warnings
from openscm_units import unit_registry  # pip install openscm-units
from openscm_twolayermodel import ImpulseResponseModel, TwoLayerModel, constants  # pip install openscm-twolayermodel
from scmdata import ScmRun  # pip install scmdata

![image.png](attachment:image.png)

In [ ]:
recommend_paras = pd.read_csv("../../data_in/recommended_irf_from_2xCO2_2021_02_25_222758.csv").iloc[0, :]
recommend_paras

In [3]:
du = (
    recommend_paras["C (W yr / m^2 / K)"] 
    * unit_registry("W yr / m^2 / delta_degC")
    / constants.DENSITY_WATER
    / constants.HEAT_CAPACITY_WATER
).to("m")
display(du)

dl = (
    recommend_paras["C_d (W yr / m^2 / K)"] 
    * unit_registry("W yr / m^2 / delta_degC")
    / constants.DENSITY_WATER
    / constants.HEAT_CAPACITY_WATER
).to("m")
display(dl)

57.739533588502034 <Unit('meter')>

1110.8078463278985 <Unit('meter')>

In [ ]:
AR6_forcing = pd.read_csv('../../data_in//AR6_ERF_1750-2019.csv', index_col=0)

In [5]:
ch6_forcing = pd.read_csv('/Users/earpmf/OneDrive - University of Leeds/PYTHON/AR6_CH6_RCMIPFIGS-fig_6.12_contrib_ERF_GSAT_hist/ar6_ch6_rcmipfigs/data_out/historic_delta_GSAT/hist_ERF_est.csv', index_col=0)

In [6]:
ch6_GSAT = pd.read_csv('/Users/earpmf/OneDrive - University of Leeds/PYTHON/AR6_CH6_RCMIPFIGS-fig_6.12_contrib_ERF_GSAT_hist/ar6_ch6_rcmipfigs/data_out/historic_delta_GSAT/Delta_T_timeseriies.csv', index_col=0)

In [7]:
ch6_forcing

,CO2,N2O,CH4,NOx,SO2,BC,OC,NH3,VOC,HC
1750,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000
1751,0.001126,0.000077,0.000832,-0.000001,-0.000053,0.000014,-0.000119,-0.000005,0.000177,0.000000
1752,0.002252,0.000154,0.001663,-0.000002,-0.000091,0.000028,-0.000237,-0.000010,0.000354,0.000000
1753,0.003378,0.000230,0.002495,-0.000004,-0.000124,0.000043,-0.000356,-0.000014,0.000530,0.000000
1754,0.004505,0.000307,0.003326,-0.000005,-0.000160,0.000057,-0.000474,-0.000019,0.000706,0.000000
...,...,...,...,...,...,...,...,...,...,...
2015,1.894420,0.223577,1.171873,-0.295162,-0.994594,0.065084,-0.201918,-0.029726,0.481837,0.200667
2016,1.948068,0.226732,1.180530,-0.291303,-0.953730,0.063885,-0.202334,-0.030035,0.464815,0.201919
2017,1.981696,0.230251,1.188029,-0.290949,-0.918519,0.063844,-0.202386,-0.030378,0.458057,0.203356
2018,2.019078,0.234670,1.197088,-0.288873,-0.913279,0.063397,-0.200986,-0.030660,0.452975,0.204997


In [ ]:
ch6_GSAT

In [ ]:
impulse_response = ImpulseResponseModel(
    d1=recommend_paras["d1 (yr)"] * unit_registry("yr"),
    d2=recommend_paras["d2 (yr)"] * unit_registry("yr"),
    q1=recommend_paras["q1 (K / (W / m^2))"] * unit_registry("delta_degC / (W / m^2)"),
    q2=recommend_paras["q2 (K / (W / m^2))"] * unit_registry("delta_degC / (W / m^2)"),
    efficacy=recommend_paras["efficacy (dimensionless)"] * unit_registry("dimensionless"),
)
res_impulse_response = impulse_response.run_scenarios()

In [28]:
erf = ch6_forcing['CO2']
scenario = "chapter 6 CO2"
unit = "W/m^2"

driver = ScmRun(
    data=erf,
    index=1750 + np.arange(len(erf)),
    columns={
        "unit": unit,
        "model": "custom",
        "scenario": scenario,
        "region": "World",
        "variable": "Effective Radiative Forcing",
    },
)
driver

impulse_response = ImpulseResponseModel(
    d1=recommend_paras["d1 (yr)"] * unit_registry("yr"),
    d2=recommend_paras["d2 (yr)"] * unit_registry("yr"),
    q1=recommend_paras["q1 (K / (W / m^2))"] * unit_registry("delta_degC / (W / m^2)"),
    q2=recommend_paras["q2 (K / (W / m^2))"] * unit_registry("delta_degC / (W / m^2)"),
    efficacy=recommend_paras["efficacy (dimensionless)"] * unit_registry("dimensionless"),
)
res_impulse_response = impulse_response.run_scenarios(driver)

two_layer = TwoLayerModel(
    # the naming is confusing because I follow Geoffroy, Ch. 7 Appendix does its own thing
    # the units should be clear
    lambda0=recommend_paras["alpha (W / m^2 / K)"] * unit_registry("W/m^2/delta_degC"),
    du=du,
    dl=dl,
    a=0.0 * unit_registry("watt / delta_degree_Celsius ** 2 / meter ** 2"),
    efficacy=recommend_paras["efficacy (dimensionless)"] * unit_registry("dimensionless"),
    eta=recommend_paras["kappa (W / m^2 / K)"] * unit_registry("W / m^2 / K")
)
res_two_layer = two_layer.run_scenarios(driver)

In [ ]:
erf

In [24]:
res_impulse_response.head()

time                                                                                                                                                                                                                                                1750-01-01  \
climate_model                  d1 (a)   d2 (a)     efficacy (dimensionless) model  q1 (delta_degree_Celsius * meter ** 2 / watt) q2 (delta_degree_Celsius * meter ** 2 / watt) region run_idx scenario      unit       variable                                  
two_timescale_impulse_response 3.424102 285.003478 1.027856                 custom 0.443768                                      0.319591                                      World  0       chapter 6 CO2 W/m^2      Effective Radiative Forcing         0.0   
                                                                                                                                                                                                            delta_degC Surface Temperature|Box 1           0.0   
                                                                                                                                                                                                                       Surface Temperature|Box 2           0.0   
                                                                                                                                                                                                                       Surface Temperature                 0.0   
                                                                                                                                                                                                            W/m^2      Heat Uptake                         0.0   

time                                                                                                                                                                                                                                                1751-01-01  \
climate_model                  d1 (a)   d2 (a)     efficacy (dimensionless) model  q1 (delta_degree_Celsius * meter ** 2 / watt) q2 (delta_degree_Celsius * meter ** 2 / watt) region run_idx scenario      unit       variable                                  
two_timescale_impulse_response 3.424102 285.003478 1.027856                 custom 0.443768                                      0.319591                                      World  0       chapter 6 CO2 W/m^2      Effective Radiative Forcing    0.001126   
                                                                                                                                                                                                            delta_degC Surface Temperature|Box 1      0.000000   
                                                                                                                                                                                                                       Surface Temperature|Box 2      0.000000   
                                                                                                                                                                                                                       Surface Temperature            0.000000   
                                                                                                                                                                                                            W/m^2      Heat Uptake                    0.000000   

time                                                                                                                                                                                                                                                1752-01-01  \
climate_model                  d1 (a)   d2 (a)     efficacy (dimensionless) model  q1 (delta_degree_Celsius * meter ** 2 / watt)

In [29]:
res_two_layer.head()

time                                                                                                                                                                                                                                                                            1750-01-01  \
a (watt / delta_degree_Celsius ** 2 / meter ** 2) climate_model dl (meter)  du (meter) efficacy (dimensionless) eta (watt / kelvin / meter ** 2) lambda0 (watt / delta_degree_Celsius / meter ** 2) model  region run_idx scenario      unit       variable                                  
0.0                                               two_layer     1110.807846 57.739534  1.027856                 0.880636                         1.31                                               custom World  0       chapter 6 CO2 W/m^2      Effective Radiative Forcing         0.0   
                                                                                                                                                                                                                                        delta_degC Surface Temperature|Upper           0.0   
                                                                                                                                                                                                                                                   Surface Temperature|Lower           0.0   
                                                                                                                                                                                                                                        W/m^2      Heat Uptake                         0.0   

time                                                                                                                                                                                                                                                                            1751-01-01  \
a (watt / delta_degree_Celsius ** 2 / meter ** 2) climate_model dl (meter)  du (meter) efficacy (dimensionless) eta (watt / kelvin / meter ** 2) lambda0 (watt / delta_degree_Celsius / meter ** 2) model  region run_idx scenario      unit       variable                                  
0.0                                               two_layer     1110.807846 57.739534  1.027856                 0.880636                         1.31                                               custom World  0       chapter 6 CO2 W/m^2      Effective Radiative Forcing    0.001126   
                                                                                                                                                                                                                                        delta_degC Surface Temperature|Upper      0.000000   
                                                                                                                                                                                                                                                   Surface Temperature|Lower      0.000000   
                                                                                                                                                                                                                                        W/m^2      Heat Uptake                    0.000000   

time                                                                                                                                                                                                                                                                            1752-01-01  \
a (watt / delta_degree_Celsius ** 2 / meter ** 2) climate_model dl (meter)  du (meter) efficacy (dimensionless) eta (watt / kelvin / meter ** 2) lambda0 (watt / delta_degree_Celsius / meter ** 2) model  region run_idx scenario      unit       variable                             

In [26]:
ch6_GSAT

,CO2,N2O,CH4,NOx,SO2,BC,OC,NH3,VOC,HC
year,,,,,,,,,,
1750,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
1751,0.000147,0.000010,0.000109,-1.534412e-07,-0.000007,0.000002,-0.000016,-6.248794e-07,0.000023,0.000000
1752,0.000405,0.000028,0.000299,-4.229968e-07,-0.000017,0.000005,-0.000043,-1.717622e-06,0.000064,0.000000
1753,0.000745,0.000051,0.000550,-7.781722e-07,-0.000029,0.000009,-0.000078,-3.160974e-06,0.000117,0.000000
1754,0.001147,0.000078,0.000847,-1.198815e-06,-0.000043,0.000014,-0.000121,-4.867373e-06,0.000180,0.000000
...,...,...,...,...,...,...,...,...,...,...
2015,1.017607,0.121303,0.668071,-1.714400e-01,-0.669099,0.039113,-0.119377,-1.638171e-02,0.296524,0.108567
2016,1.039232,0.123298,0.673168,-1.701955e-01,-0.649515,0.038738,-0.119548,-1.655779e-02,0.292309,0.109317
2017,1.060235,0.125301,0.678216,-1.692870e-01,-0.630467,0.038466,-0.119725,-1.674113e-02,0.288373,0.110115


In [27]:
print (0.95/1.1)

0.8636363636363635
